# Classificador Random Forest - Implantação

Baseado no algoritmo [Árvores de Decisão](https://en.wikipedia.org/wiki/Decision_tree), o algoritmo [Random Forest](https://en.wikipedia.org/wiki/Random_forest) opera construindo múltiplas Árvores de Decisão com o intuito de combinar essas Árvores e gerar uma Árvore de Decisão final. 

<img src="https://i.imgur.com/27PbBzH.png" alt="RandomForest" width="600"/>

O algoritmo Random Forest está na classe de [Ensemble Methods](https://en.wikipedia.org/wiki/Ensemble_learning) que visa a combinação de múltiplos algoritmos de Machine Learning para obter melhor performance. O Random Forest pode fazer ótimas predições em conjuntos de dados de larga escala, e geralmente tem resultados melhores que as Árvores de Decisão.

Este componente realiza predições usando um modelo Random Forest para classificação usando [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html). <br>
Scikit-learn é uma biblioteca open source de machine learning que suporta apredizado supervisionado e não supervisionado. Também provê várias ferramentas para montagem de modelo, pré-processamento de dados, seleção e avaliação de modelos, e muitos outros utilitários.

Este notebook apresenta:
- como carregar modelos e outros resultados do treinamento.
- como utilizar um modelo para fornecer predições em tempo real.

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [ ]:
%%writefile Model.py
import joblib
import numpy as np
import pandas as pd


class Model(object):
    def __init__(self):
        # Carrega artefatos: estimador, etc
        artifacts = joblib.load("/tmp/data/random-forest-classifier.joblib")
        self.pipeline = artifacts["pipeline"]
        self.features_names_training = artifacts["columns"]
        self.columns_to_filter = artifacts["columns_to_filter"]
        self.label_encoder = artifacts["label_encoder"]
        self.method = artifacts["method"]
        self.new_columns = artifacts["new_columns"]
        self.features_after_pipeline = artifacts["features_after_pipeline"]

    def class_names(self):
        column_names = np.concatenate((self.columns_to_filter, self.new_columns))
        return column_names.tolist()

    def predict(self, X, feature_names, meta=None):
        
        df = pd.DataFrame(X)
        
        if feature_names:
            # Antes de utilizar o conjunto de dados X no modelo, reordena suas features de acordo com a ordem utilizada no treinamento
            df = pd.DataFrame(X, columns=feature_names)[self.columns_to_filter]
            X = df.to_numpy()

        # adiciona a predição ao conjunto de dados
        y_prob = self.pipeline.predict_proba(X)
        y_pred = self.pipeline.predict(X)
        y_pred = self.label_encoder.inverse_transform(y_pred)
        
        df.loc[:, self.new_columns[:-1]] = y_prob
        df.loc[:, self.new_columns[-1]] = y_pred

        return df.to_numpy()